##Ordinal Regression

In [1]:
## Imports
import matplotlib.pyplot as plt
# import scipy as sco
import autograd.scipy as sci  # Thinly-wrapped scipy
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd import grad
from sklearn.linear_model import LogisticRegression
import os
import pandas as pd
import itertools
import time

In [3]:
import findspark
findspark.init()
import pyspark
sc=pyspark.SparkContext()

### Load the data

In [4]:
df=pd.read_csv('../data/musicdata.csv',header=None)
df.columns=['uid', 'aid', 'rating']

# I and J are the number of users and artists, respectively
I = df.uid.max() + 1
J = df.aid.max() + 1



The loglikelihood is given as
$$ \mathcal{L}(\beta \; \vert \; \mathcal{D}, \theta) = \sum_{i=1}^n \log \left( F_\epsilon(\theta_{Y_i} + X_i^T \beta) - F_\epsilon(\theta_{Y_i-1} + X_i^T\beta) \right)$$

We first write a log likelihood function that calculates it given an entire array, and iterates over the rows.

In [5]:
# the log-likelihood function
# Xrat array consisting of (rating, user, item)
# Xfeat array indexable by the item column of Xrat which delivers item features
# theta is the parameter vector
# theta = (u_1, ..., u_I, v_1, ..., v_J, a_1, ..., a_I, b_1, ..., b_J, g, beta_1, ..., beta_L)
# i.e. theta has length (I + J) * K + I + J + 1 + L
# with u_i, v_j \in \mathbb{R}^K
# b stores the buckets for R categories (i.e. we have R-1 elements in b)
# i.e. in b we store b_1, b_2, ..., b_{R-1} for R different categories
def loglikelihood(theta, Xrat, b, I, J, K, R):
    
    
    # first implement the easy latent model using probit...
    llsum = 0
    for row in xrange(Xrat.shape[0]):
        rating = Xrat[row, 0]
        i = Xrat[row, 1]
        j = Xrat[row, 2]

        # asserts for the indices i, j & rating
        assert i < I and i >= 0
        assert j < J and j >= 0
        assert rating <= R

        # the model for the latent variable
        u_i = theta[K * i:K*(i+1)]
        v_j = theta[(I + j) * K:(I + j + 1) * K]
        a_i = theta[(I + J) * K + i]
        b_j = theta[(I + J) * K + I + j]
        g = theta[(I + J) * K + I + J]
        beta = theta[(I + J) * K + I + J + 1:]

        # some asserts for the sizes
        assert len(u_i) == K
        assert len(v_j) == K

        # the full model (reduce if necessary)
        # model with features does not work yet...
#         model = np.dot(u_i, v_j) + a_i + b_j + g + np.dot(Xfeat[j, :], beta) 

        # here are some other choices

        # easy model using features only
        # model = np.dot(Xfeat[j, :], beta)

        # model using features + biases
        # model = np.dot(Xfeat[j, :], beta) + a_i + b_j + g

        # model using latent factors for user/item
#         model = np.dot(u_i, v_j)

        # model using latent factors for user/item and biases
        model = np.dot(u_i, v_j) + a_i + b_j + g

        # the ordinal regression part
        # if rating is 1 or R we have a special case
        # another possibility would be to use instead of the ifelse construction
        # dummy values like +/- 9999 for infty
        # note the additional -1 due to space saving!
        if rating == R:
            # note F(infty) = 0 (mathematically not rigourous, limit is more correct)
            llsum += np.log(1 - sci.stats.norm.cdf(b[rating - 2] + model))
        elif rating == 1:
            # note F(-infty) = 0
            llsum += np.log(sci.stats.norm.cdf(b[rating - 1] + model))
        else:
            llsum += np.log(sci.stats.norm.cdf(b[rating - 1] + model) - sci.stats.norm.cdf(b[rating - 2] + model))
    return llsum

We then define a gradient function, in the same manner, that calculates the gradient row by row. 

In [6]:
# by hand derived gradient
def manual_grad(theta, Xrat, b, I, J, K, R):
    grad = np.zeros(theta.shape[0])
    
    # go over samples and make update according to them!
    for n in xrange(Xrat.shape[0]):
        
        rating = Xrat[n, 0]
        i = Xrat[n, 1]
        j = Xrat[n, 2]
        
        # the model for the latent variable
        u_i = theta[K * i:K*(i+1)]
        v_j = theta[(I + j) * K:(I + j + 1) * K]
        a_i = theta[(I + J) * K + i]
        b_j = theta[(I + J) * K + I + j]
        g = theta[(I + J) * K + I + J]
        beta = theta[(I + J) * K + I + J + 1:]

        # some asserts for the sizes
        assert len(u_i) == K
        assert len(v_j) == K
        
        model = np.dot(u_i, v_j)
        if rating == R:
            q = 1 - sc.stats.norm.pdf(b[rating - 2] + model) / (1 - sc.stats.norm.cdf(b[rating - 2] + model))
        elif rating == 1:
            q = sc.stats.norm.pdf(b[rating - 1] + model) / sc.stats.norm.cdf(b[rating - 1] + model)
        else:
            q = sc.stats.norm.pdf(b[rating-2] + model) - sc.stats.norm.pdf(b[rating - 1] + model) / (sc.stats.norm.cdf(b[rating-2] + model) - sc.stats.norm.cdf(b[rating - 1] + model))
        
        # compute derivatives
        grad_u_i = q * v_j
        grad_v_j = q * u_i
        grad_a_i = q 
        grad_b_j = q
        grad_g = q
#         grad_beta = q * Xfeat[j, :]
        
        grad[K * i:K*(i+1)] += grad_u_i
        grad[(I + j) * K:(I + j + 1) * K] += grad_v_j
        grad[(I + J) * K + i] += grad_a_i
        grad[(I + J) * K + I + j] += grad_b_j
        grad[(I + J) * K + I + J] += grad_g
#         grad[(I + J) * K + I + J + 1:] += grad_beta
        
    return grad

The data is then prepped. We begin with a random theta vector, and calculate the log likelihood to ensure that it is working correctly.

In [7]:

# create sample df
dftouse = df[['rating', 'uid', 'aid']].head(2000)
dftouse.rating=np.around((dftouse.rating-1)/20)
# as in the given dataset the indices are 1,...,I and 1, ..., J
# adjust jokes & uid s.t. they serve the 0, ..., I-1 and 0, ..., J-1 space 
dftouse['uid'] = dftouse['uid'] - 1
dftouse['aid'] = dftouse['aid'] - 1

# transform ratings to range 1, ..., R
rating_vals = np.arange(1,dftouse.rating.max()+1)
minR = dftouse.rating.min()
dftouse['rating'] = dftouse['rating'] - minR
R = len(rating_vals)


# create buckets as midpoints
buckets = 0.5 * (rating_vals[1:] + rating_vals[:-1])

# get length I, J
I = dftouse.uid.max() + 1
J = dftouse.aid.max() + 1

# define some K
K = 2

# convert to numpy data matrix
Xrat = np.array(dftouse)
# Xfeat = Jf

# create dummy theta vector (all zeros)
# L = Xfeat.shape[1]
theta = np.zeros((I + J) * K + I + J + 1)

# init theta vector with some random values
theta = np.random.normal(size=theta.shape[0], loc=0., scale=0.1)

# compute log likelihood
loglikelihood(theta, Xrat, buckets, I, J, K, R)

-16018.296162212204

The following function calculates the likelihood like before, but this time only for a single row. 

In [8]:
## the rowlikelihood for sgd
def rowloglikelihood(theta, row, b, I, J, K, R):
    rating = row[0]
    i = row[1]
    j = row[2]

    # asserts for the indices i, j & rating
    assert i < I and i >= 0
    assert j < J and j >= 0
    assert rating <= R

    # the model for the latent variable
    u_i = theta[K * i:K*(i+1)]
    v_j = theta[(I + j) * K:(I + j + 1) * K]
    a_i = theta[(I + J) * K + i]
    b_j = theta[(I + J) * K + I + j]
    g = theta[(I + J) * K + I + J]
    beta = theta[(I + J) * K + I + J + 1:]

    # some asserts for the sizes
    assert len(u_i) == K
    assert len(v_j) == K

    # the full model (reduce if necessary)
    # model with features does not work yet...
#     model = np.dot(u_i, v_j) + a_i + b_j + g + np.dot(Xfeat[j, :], beta) 
    
    # here are some other choices
    
    # easy model using features only
    # model = np.dot(Xfeat[j, :], beta)
    
    # model using features + biases
    # model = np.dot(Xfeat[j, :], beta) + a_i + b_j + g
    
    # model using latent factors for user/item
#     model = np.dot(u_i, v_j)
    
    # model using latent factors for user/item and biases
    model = np.dot(u_i, v_j) + a_i + b_j + g

    # the ordinal regression part
    # if rating is 1 or R we have a special case
    # another possibility would be to use instead of the ifelse construction
    # dummy values like +/- 9999 for infty
    # note the additional -1 due to space saving!
    if rating == R:
        # note F(infty) = 0 (mathematically not rigourous, limit is more correct)
        return np.log(1 - sci.stats.norm.cdf(b[rating - 2] + model))
    elif rating == 1:
        # note F(-infty) = 0
        return np.log(sci.stats.norm.cdf(b[rating - 1] + model))
    else:
        return np.log(sci.stats.norm.cdf(b[rating - 1] + model) - \
                      sci.stats.norm.cdf(b[rating - 2] + model))

Along the same vein, we calculate the gradient for a single row. 

In [19]:
# by hand derived gradient
def row_manual_grad(theta, row, b, I, J, K, R):
    grad = np.zeros(theta.shape[0])
    

    rating = row[0]
    i = row[1]
    j = row[2]

    # the model for the latent variable
    u_i = theta[K * i:K*(i+1)]
    v_j = theta[(I + j) * K:(I + j + 1) * K]
    a_i = theta[(I + J) * K + i]
    b_j = theta[(I + J) * K + I + j]
    g = theta[(I + J) * K + I + J]
    beta = theta[(I + J) * K + I + J + 1:]

    # some asserts for the sizes
    assert len(u_i) == K
    assert len(v_j) == K

    model = np.dot(u_i, v_j)
    if rating == R:
        q = 1 - sci.stats.norm.pdf(b[rating - 2] + model) / (1 - sci.stats.norm.cdf(b[rating - 2] + model))
    elif rating == 1:
        q = sci.stats.norm.pdf(b[rating - 1] + model) / sci.stats.norm.cdf(b[rating - 1] + model)
    else:
        q = sci.stats.norm.pdf(b[rating-2] + model) - sci.stats.norm.pdf(b[rating - 1] + model) / (sci.stats.norm.cdf(b[rating-2] + model) - sci.stats.norm.cdf(b[rating - 1] + model))

    # compute derivatives
    grad_u_i = q * v_j
    grad_v_j = q * u_i
    grad_a_i = q 
    grad_b_j = q
    grad_g = q
#         grad_beta = q * Xfeat[j, :]

    grad[K * i:K*(i+1)] = grad_u_i
    grad[(I + j) * K:(I + j + 1) * K] = grad_v_j
    grad[(I + J) * K + i] = grad_a_i
    grad[(I + J) * K + I + j] = grad_b_j
    grad[(I + J) * K + I + J] = grad_g
#         grad[(I + J) * K + I + J + 1:] += grad_beta

    return grad

The two functions below use the autograd library to get the gradient of our log likelihood functions.

In [14]:
gradrll = grad(rowloglikelihood)
gradll = grad(loglikelihood)

Now we get a benchmark for how fast it runs serially, using the gradient we calculated manually.

In [47]:
# one epoch of sgd!

theta0 = theta.copy()

# combined learning_rate * scale_factor
alpha = 0.05

t=time.time()
for row in Xrat:
    # update theta0 according to current row
    theta0 += alpha * row_manual_grad(theta0, row, buckets, I, J, K, R)
print theta0

print "Time taken: %f s" % (time.time()-t)
print loglikelihood(theta0, Xrat, buckets, I, J, K, R)

[  0.05495333  -0.05574243  -0.07264744 ...,  -0.03478058  -0.0502744
 -34.01143916]
Time taken: 58.311484 s
-834201.329185


### Parallelization 

Parallelize! Stick Xrat into an RDD, copy over the theta from before. 


In [20]:
xrat_rdd=sc.parallelize(Xrat)

# Get a fresh theta
theta1=theta.copy()
file=open("out.txt","w")
#And then compute the gradient for each row individually, and the sum it
t=time.time()
ptheta1=xrat_rdd.map(lambda x: alpha*row_manual_grad(theta1,x,buckets, I, J, K, R)).mean()
tot=time.time()-t
file.write(tot+"\n")
file.write(loglikelihood(ptheta1, Xrat, buckets, I, J, K, R))
file.close()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:703)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:702)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:702)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1514)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1438)
	at org.apache.spark.SparkContext$$anonfun$stop$7.apply$mcV$sp(SparkContext.scala:1724)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1185)
	at org.apache.spark.SparkContext.stop(SparkCon

That log likelihood is pretty awful though. Lets test it by breaking it up into subarrays of size S and try again.

In [242]:
# A function we pass to the array to calculate the updated thetas from each subarray

def n_row_sgd(theta,subX, buckets, I, J, K, R):
    for row in subX:
        # update theta0 according to current row
        theta += alpha * gradrll(theta, row, buckets, I, J, K, R)
    return theta

In [244]:
S=20
#Split the array into subarrays of size n
split_xrat=np.split(Xrat,Xrat.shape[0]/size)

#And then parallelize it
split_xrat = sc.parallelize(split_xrat,5)

# Get a fresh theta
theta2=theta.copy()

# Run the sgd!

ptheta2=split_xrat.map(lambda subX:n_row_sgd(theta2, subX, buckets, I, J, K, R)).mean()

OSError: [Errno 2] No such file or directory: '/private/var/folders/hm/mnf_gkh50zv3ptc6xqchtyh40000gn/T/spark-a3b0825c-198a-4a2d-8056-ba71ba6bcb4d/pyspark-06f5b308-c291-49d0-8c36-61a3f9784d3e/tmptJURPL'

In [ ]:
# # Overall Todos:
# # add Documentation, code in MLib style

# # This shall be designed for modelling ratings
# # assume we are given n users i =1, ..., I
# # that rated m items j= 1, ..., J
# # Y_n ~ u_i^Tv_j + a_i + b_j + g + X_n * w
# # u_i, v_j are K-dimensional latent variable vectors
# # a_i, b_j, g are user/item/global wise biases (latent)
# # X_n represents the n-th data row with L features
# # w is the weight vector we want to train for
# # Thus, in total our model trains the parameter vector
# # theta = (u_1, ..., u_I, v_1, ..., v_J, a_1, ..., a_I, b_1, ..., b_J, g, w)
# class CumulativeModel:
    
    
#     def __init__(self, buckets = None, modelType = 'logistic', spark=False, numLatentFactors=0):
#         # add here check for correct model types
#         # logistic, probit, minev, maxev
#         self.mtype = modelType
#         self.useSpark = spark
#         self.K = numLatentFactors
#         self.buckets = buckets
        
#     # add here sgd parameters
#     def fit(self, X, y, ...):
#         # first set buckets if necessary
#         if self.buckets is None:
#             warning()
        
#     # make a prediction
#     def predict(self, X, y, ...):
        